In [ ]:
import os
import gpxpy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from pyproj import Transformer
import contextily as ctx

# Set backend
plt.switch_backend('TkAgg')

folder_path = r'D:\Python\gps'
selected_files = ['track1.gpx', 'track2.gpx']
colors = ['red', 'blue']

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

tracks_xy = []
labels = []

for idx, filename in enumerate(selected_files):
    full_path = os.path.join(folder_path, filename)
    if os.path.isfile(full_path):
        with open(full_path, 'r') as f:
            gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                points = [(point.longitude, point.latitude) for point in segment.points]
                if len(points) > 1:
                    x, y = transformer.transform(
                        [pt[0] for pt in points],
                        [pt[1] for pt in points]
                    )
                    tracks_xy.append((x, y, colors[idx], filename))
                    labels.append(filename)
        print(f"Loaded {filename}: {len(points)} points")
    else:
        print(f"{filename} not found!")

if tracks_xy:
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Calculate bounds
    all_x = [x for track in tracks_xy for x in track[0]]
    all_y = [y for track in tracks_xy for y in track[1]]
    
    minx, maxx = min(all_x), max(all_x)
    miny, maxy = min(all_y), max(all_y)
    
    # Add some padding around the tracks
    x_padding = (maxx - minx) * 0.1
    y_padding = (maxy - miny) * 0.1
    
    ax.set_xlim(minx - x_padding, maxx + x_padding)
    ax.set_ylim(miny - y_padding, maxy + y_padding)
    
    # ADD BASEMAP FIRST (before plotting tracks)
    print("Adding basemap...")
    try:
        ctx.add_basemap(ax, crs='EPSG:3857', source=ctx.providers.OpenStreetMap.Mapnik)
        print("Basemap added successfully")
    except Exception as e:
        print(f"Basemap error: {e}")
        # Fallback to simpler basemap
        try:
            ctx.add_basemap(ax, crs='EPSG:3857')
            print("Fallback basemap added")
        except:
            print("Could not add basemap, continuing without it")
    
    # Create animated elements
    lines = []
    points = []
    
    for idx, (x, y, color, label) in enumerate(tracks_xy):
        # Static background (faint)
        ax.plot(x, y, color=color, alpha=0.3, linewidth=4, label=label)
        # Animated line (bold)
        line, = ax.plot([], [], color=color, linewidth=6, alpha=0.9)
        lines.append(line)
        # Moving point
        point, = ax.plot([], [], 'o', color=color, markersize=14, 
                        markeredgecolor='white', markeredgewidth=2, zorder=5)
        points.append(point)
    
    ax.legend()
    ax.set_title("Animated Jogging Routes with Basemap")
    ax.axis('off')  # Turn off axes for cleaner look with basemap
    
    # Animation
    max_len = max(len(x) for x, y, _, _ in tracks_xy)
    
    def animate(frame):
        for idx, (x, y, color, _) in enumerate(tracks_xy):
            progress = min(frame, len(x))
            lines[idx].set_data(x[:progress], y[:progress])
            if progress > 0:
                points[idx].set_data([x[progress-1]], [y[progress-1]])
        return lines + points
    
    anim = FuncAnimation(fig, animate, frames=max_len, interval=50, blit=True, repeat=True)
    
    print("Showing animation with basemap...")
    plt.tight_layout()
    plt.show()

else:
    print("No tracks found")